In [3]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.utils import image_dataset_from_directory
import pathlib

# Transfer learning with Xception Model

In [26]:
data_folder = pathlib.Path('../../facial_expression_recognition/images')
target_img_size = (71, 71)

train_dataset = image_dataset_from_directory(
    data_folder / "train",
    image_size= target_img_size,
    label_mode='categorical',
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    data_folder / "validation",
    image_size= target_img_size,
    label_mode='categorical',
    batch_size=32)

Found 28821 files belonging to 7 classes.
Found 7066 files belonging to 7 classes.


In [33]:
for images, labels in train_dataset:
    images = np.stack((images, ) * 3, axis=-1)


In [35]:
for images, labels in validation_dataset:
   images = np.stack((images, ) * 3, axis=-1)

In [36]:
conv_base = keras.applications.Xception(
    weights="imagenet",
    include_top=False)

In [37]:

# Freeze all layers
conv_base.trainable = False
conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   864         ['input_5[0][0]']                
                                32)                                                               
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, None, None,   128        ['block1_conv1[0][0]']           
 ation)                         32)                                                        

In [38]:
from keras import layers

inputs = keras.Input(shape=(71, 71, 3))
x = tf.keras.applications.xception.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(7, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [39]:
opt = keras.optimizers.Nadam(learning_rate=1e-5)
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="./models/xception_transfer_image_resized.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/50
901/901 [==============================] - 21s 17ms/step - loss: 1.7274 - accuracy: 0.3522 - val_loss: 1.5322 - val_accuracy: 0.4236
Epoch 2/50
901/901 [==============================] - 15s 17ms/step - loss: 1.5445 - accuracy: 0.4179 - val_loss: 1.4938 - val_accuracy: 0.4410
Epoch 3/50
901/901 [==============================] - 15s 16ms/step - loss: 1.4751 - accuracy: 0.4427 - val_loss: 1.4535 - val_accuracy: 0.4533
Epoch 4/50
901/901 [==============================] - 14s 16ms/step - loss: 1.4299 - accuracy: 0.4638 - val_loss: 1.4709 - val_accuracy: 0.4502
Epoch 5/50
901/901 [==============================] - 14s 16ms/step - loss: 1.3909 - accuracy: 0.4766 - val_loss: 1.4500 - val_accuracy: 0.4605
Epoch 6/50
901/901 [==============================] - 15s 16ms/step - loss: 1.3688 - accuracy: 0.4851 - val_loss: 1.4336 - val_accuracy: 0.4680
Epoch 7/50
901/901 [==============================] - 14s 16ms/step - loss: 1.3459 - accuracy: 0.4935 - val_loss: 1.4402 - val_accuracy:

## Use Xception Model with freezing top 2 layers, fine tuning